# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [ ]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Copy Schema to a dataframe

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'mgmt467project1' # This is derived from your provided table name
dataset_id = 'lab1dataset'
table_id = 'SuperStore'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


Schema DataFrame created:


## CLean Column Names

In [ ]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `mgmt467project1.lab1dataset.superstore_clean` AS
SELECT
  `Row ID` AS row_id,
  `Order ID` AS order_id,
  `Order Date` AS order_date,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Segment AS segment,
  Country AS country,
  City AS city,
  State AS state,
  `Postal Code` AS postal_code,
  Region AS region,
  `Product ID` AS product_id,
  Category AS category,
  `Sub-Category` AS sub_category,
  `Product Name` AS product_name,
  Sales AS sales,
  Quantity AS quantity,
  Discount AS discount,
  Profit AS profit
FROM
  `mgmt467project1.lab1dataset.SuperStore`;



## Generate View with standard column naming convention

In [ ]:
try:
    # Query the view directly
    sql_str = f"""
    SELECT *
    FROM `{project_id}.{dataset_id}.{new_view_id}`
    LIMIT 10
    """

    df_preview = client.query(sql_str).to_dataframe()

    # Print header
    print(" | ".join(df_preview.columns))
    print("-" * 80)  # Separator

    # Print rows
    for _, row in df_preview.iterrows():
        print(" | ".join([str(item) for item in row.values]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



row_id | order_id | order_date | ship_date | ship_mode | customer_id | customer_name | segment | country | city | state | postal_code | region | product_id | category | sub_category | product_name | sales | quantity | discount | profit
--------------------------------------------------------------------------------
5769 | CA-2015-154900 | 2015-02-25 | 2015-03-01 | Standard Class | SS-20875 | Sung Shariari | Consumer | United States | Leominster | Massachusetts | 1453 | East | OFF-LA-10001641 | Office Supplies | Labels | Avery 518 | 3.15 | 1 | 0.0 | 1.512
5770 | CA-2015-154900 | 2015-02-25 | 2015-03-01 | Standard Class | SS-20875 | Sung Shariari | Consumer | United States | Leominster | Massachusetts | 1453 | East | OFF-PA-10002377 | Office Supplies | Paper | Adams Telephone Message Book W/Dividers/Space For Phone Numbers, 5 1/4"X8 1/2", 200/Messages | 22.72 | 4 | 0.0 | 10.224
9028 | US-2016-152415 | 2016-09-17 | 2016-09-22 | Standard Class | PO-18865 | Patrick O'Donnell | Consumer | Un

In [ ]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `mgmt467project1.lab1dataset.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,order_id,customer_name,product_name,sales,profit
0,CA-2015-154900,Sung Shariari,Avery 518,3.15,1.5120
1,CA-2015-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2016-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2016-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,CA-2016-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
5,CA-2016-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
6,CA-2016-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,CA-2016-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
8,CA-2015-158792,Brian Dahlen,Staples,22.20,10.4340
9,CA-2016-141082,Fred McMath,Avery 517,3.69,1.7343


## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `mgmt-467-47888.lab1_foundation.superstore`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

I feel like the results matched my expectations but at first i didnt run the query through here so it showed different tables.

In [ ]:
query_string = """
SELECT DISTINCT Sub_Category
FROM `mgmt467project1.lab1dataset.superstore_clean`
WHERE Region = 'West'
ORDER BY Sub_Category ASC
LIMIT 100
"""

# Run the query
query_job = client.query(query_string)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)


,Sub_Category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [ ]:
query_string = """
SELECT
  Customer_ID,
  SUM(Profit) AS total_profit
FROM `mgmt467project1.lab1dataset.superstore_clean`
GROUP BY Customer_ID
ORDER BY total_profit DESC
LIMIT 10;
"""

# Run the query
query_job = client.query(query_string)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)

,Customer_ID,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055
5,TA-21385,4703.7883
6,CM-12385,3899.8904
7,KD-16495,3038.6254
8,AR-10540,2884.6208
9,DR-12940,2869.0760


The top 3 customers by total profit were TC-20980, RB-19360, and SC-20095.

### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

In [ ]:

query_string = """
SELECT
  Ship_Mode,
  COUNT(*) AS order_count
FROM `mgmt467project1.lab1dataset.superstore_clean`
WHERE Category = 'Technology'
GROUP BY Ship_Mode
ORDER BY order_count DESC;

"""

# Run the query
query_job = client.query(query_string)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)

,Ship_Mode,order_count
0,Standard Class,1082
1,Second Class,366
2,First Class,301
3,Same Day,98


The order counts by shipping mode for Technology category products show standard class was the ship mode used most frequently.

## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [ ]:

query_string = """
WITH bounds AS (
  SELECT DATE_TRUNC(MAX(DATE(Order_Date)), MONTH) AS anchor_month
  FROM `mgmt467project1.lab1dataset.superstore_clean`
)
SELECT
  FORMAT_DATE('%Y-%m', DATE_TRUNC(DATE(Order_Date), MONTH)) AS year_month,
  SUM(Sales) AS monthly_revenue
FROM `mgmt467project1.lab1dataset.superstore_clean`
CROSS JOIN bounds
WHERE DATE(Order_Date) >= DATE_SUB(anchor_month, INTERVAL 12 MONTH)
  AND DATE(Order_Date) <  anchor_month
GROUP BY year_month
ORDER BY year_month ASC
LIMIT 100;


"""

# Run the query
query_job = client.query(query_string)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)


,year_month,monthly_revenue
0,2016-12,96999.0430
1,2017-01,43971.3740
2,2017-02,20301.1334
3,2017-03,58872.3528
4,2017-04,36521.5361
5,2017-05,44261.1102
6,2017-06,52981.7257
7,2017-07,45264.4160
8,2017-08,63120.8880
9,2017-09,87866.6520


This query calculates the total sales revenue per month for the most recent 12 full months available in your dataset.

### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

In [29]:

query_string = """
SELECT
  Sub_Category,
  SUM(Profit) AS total_profit
FROM `mgmt467project1.lab1dataset.superstore_clean`
GROUP BY Sub_Category
HAVING SUM(Profit) < 0
ORDER BY total_profit ASC;


"""

# Run the query
query_job = client.query(query_string)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)

,Sub_Category,total_profit
0,Tables,-17725.4811
1,Bookcases,-3472.5560
2,Supplies,-1189.0995


HAVING is used instead of WHERE because the filter is applied to the aggregated value (SUM(Profit)), which is only available after grouping.

## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [32]:


query_string = """
WITH products AS (
  SELECT
    Product_ID,
    ANY_VALUE(Product_Name) AS Product_Name
  FROM `mgmt467project1.lab1dataset.superstore_clean`
  WHERE Product_ID IN (
    'FUR-BO-10000112',
    'FUR-BO-10000330',
    'FUR-BO-10000362',
    'FUR-BO-10000468',
    'FUR-BO-10000711',
    'FUR-BO-10000780',
    'FUR-BO-10001519',
    'FUR-BO-10001567',
    'FUR-BO-10001601'
  )
  GROUP BY Product_ID
)
SELECT
  s.Product_ID,
  p.Product_Name,
  SUM(s.Sales) AS total_sales
FROM `mgmt467project1.lab1dataset.superstore_clean` AS s
JOIN products AS p
USING (Product_ID)
GROUP BY s.Product_ID, p.Product_Name
ORDER BY total_sales DESC;

"""

# Run the query
query_job = client.query(query_string)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)


,Product_ID,Product_Name,total_sales
0,FUR-BO-10000780,O'Sullivan Plantations 2-Door Library in Landv...,2946.3668
1,FUR-BO-10000362,Sauder Inglewood Library Bookcases,2154.3480
2,FUR-BO-10001519,O'Sullivan 3-Shelf Heavy-Duty Bookcases,1119.1950
3,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",1064.6240
4,FUR-BO-10001601,"Sauder Mission Library with Doors, Fruitwood F...",935.1972
5,FUR-BO-10000711,"Hon Metal Bookcases, Gray",851.7600
6,FUR-BO-10000112,"Bush Birmingham Collection Bookcase, Dark Cherry",825.1740
7,FUR-BO-10000468,O'Sullivan 2-Shelf Heavy-Duty Bookcases,723.8420
8,FUR-BO-10001567,"Bush Westfield Collection Bookcases, Dark Cher...",90.8820


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

In [33]:
query_string = """

WITH state_sales AS (
  SELECT
    Region,
    State,
    SUM(Sales) AS total_sales
  FROM `mgmt467project1.lab1dataset.superstore_clean`
  GROUP BY Region, State
),
ranked_state_sales AS (
  SELECT
    Region,
    State,
    total_sales,
    RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS sales_rank
  FROM state_sales
)
SELECT
  Region,
  State,
  total_sales,
  sales_rank
FROM ranked_state_sales
WHERE sales_rank <= 3
ORDER BY Region, sales_rank;

"""

# Run the query
query_job = client.query(query_string)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)




,Region,State,total_sales,sales_rank
0,Central,Texas,170188.0458,1
1,Central,Illinois,80166.1010,2
2,Central,Michigan,76269.6140,3
3,East,New York,310876.2710,1
4,East,Pennsylvania,116511.9140,2
5,East,Ohio,78258.1360,3
6,South,Florida,89473.7080,1
7,South,Virginia,70636.7200,2
8,South,North Carolina,55603.1640,3
9,West,California,457687.6315,1


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

In [35]:
query_string = """

WITH yr_sales AS (
  SELECT
    Sub_Category,
    EXTRACT(YEAR FROM DATE(Order_Date)) AS yr,
    SUM(Sales) AS total_sales
  FROM `mgmt467project1.lab1dataset.superstore_clean`
  GROUP BY Sub_Category, yr
)
SELECT
  Sub_Category,
  MAX(IF(yr = 2016, total_sales, NULL)) AS sales_2016,
  MAX(IF(yr = 2017, total_sales, NULL)) AS sales_2017,
  (MAX(IF(yr = 2017, total_sales, NULL)) - MAX(IF(yr = 2016, total_sales, NULL))) AS yoy_delta
FROM yr_sales
GROUP BY Sub_Category
HAVING sales_2016 IS NOT NULL AND sales_2017 IS NOT NULL
ORDER BY yoy_delta DESC
LIMIT 5;

"""

# Run the query
query_job = client.query(query_string)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)

,Sub_Category,sales_2016,sales_2017,yoy_delta
0,Phones,78962.030,105340.516,26378.486
1,Binders,49683.325,72788.045,23104.720
2,Accessories,41895.854,59946.232,18050.378
3,Appliances,26050.315,42926.932,16876.617
4,Copiers,49599.410,62899.388,13299.978


One common failure mode in a year-over-year (YoY) query is when one of the target years is missing from the dataset. For example, if your data contains sales for 2016 but nothing for 2017, the HAVING filter requiring both years will cause the query to return no rows at all. The simplest way to handle this is to replace the HAVING filter with COALESCE in the select list, which will treat missing values as 0 instead of excluding them. This ensures you still see results for sub-categories, even if one year’s data is absent.

Another failure mode occurs when there is uneven or partial year coverage. For instance, if one year only has data for January through March while the comparison year has a full twelve months, the YoY delta will be misleadingly large or small. To handle this, it’s important to validate the completeness of each year before running the comparison. A quick check is to count the number of distinct months per year; if a year has fewer than twelve, you can either exclude it from analysis or clearly flag it in reporting so stakeholders don’t misinterpret the results.

## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

In [36]:
query_string = """

WITH subcat_sales AS (
  SELECT
    Region,
    Sub_Category,
    SUM(Sales) AS total_sales
  FROM mgmt467project1.lab1dataset.superstore_clean
  GROUP BY Region, Sub_Category
),
ranked AS (
  SELECT
    Region,
    Sub_Category,
    total_sales,
    ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS rn
  FROM subcat_sales
)
SELECT
  Region,
  Sub_Category,
  total_sales
FROM ranked
WHERE rn = 1
ORDER BY Region;

"""

# Run the query
query_job = client.query(query_string)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)



,Region,Sub_Category,total_sales
0,Central,Chairs,85230.646
1,East,Phones,100614.982
2,South,Phones,58304.438
3,West,Chairs,101781.328


ROW_NUMBER() guarantees exactly one “winner” per region even if there’s a tie, because it assigns a unique sequential number to each row. By contrast, RANK() assigns the same rank to tied rows, which could return multiple sub-categories tied for first place in a region instead of just one.

### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

In [37]:
query_string = """

WITH subcat_sales AS (
  SELECT
    EXTRACT(YEAR FROM DATE(Order_Date)) AS yr,
    SUM(Sales) AS yearly_revenue
  FROM `mgmt467project1.lab1dataset.superstore_clean`
  WHERE Sub_Category = 'Phones'
  GROUP BY yr
),
ranked AS (
  SELECT
    yr AS year,
    yearly_revenue,
    LAG(yearly_revenue) OVER (ORDER BY yr) AS prev_revenue
  FROM subcat_sales
)
SELECT
  year,
  yearly_revenue,
  prev_revenue,
  CASE
    WHEN prev_revenue IS NULL OR prev_revenue = 0 THEN NULL
    ELSE 100.0 * (yearly_revenue - prev_revenue) / prev_revenue
  END AS yoy_pct
FROM ranked
ORDER BY year ASC;

"""

# Run the query
query_job = client.query(query_string)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)


,year,yearly_revenue,prev_revenue,yoy_pct
0,2014,77390.806,NaN,NaN
1,2015,68313.702,77390.806,-11.728918
2,2016,78962.030,68313.702,15.587397
3,2017,105340.516,78962.030,33.406545


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

In [38]:
query_string = """

WITH monthly AS (
  SELECT
    DATE_TRUNC(DATE(Order_Date), MONTH) AS month,
    SUM(Sales) AS monthly_revenue
  FROM `mgmt467project1.lab1dataset.superstore_clean`
  WHERE Segment = 'Corporate'
  GROUP BY month
)
SELECT
  month,
  monthly_revenue,
  AVG(monthly_revenue) OVER (
    ORDER BY month
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
  ) AS ma_3
FROM monthly
ORDER BY month ASC;

"""

# Run the query
query_job = client.query(query_string)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)

# 💡 Cost control tip: add a date filter (e.g., WHERE Order_Date >= '2017-01-01') while iterating.


,month,monthly_revenue,ma_3
0,2014-01-01,1701.5280,1701.528000
1,2014-02-01,1183.6680,1442.598000
2,2014-03-01,11106.7990,4663.998333
3,2014-04-01,14131.7290,8807.398667
4,2014-05-01,9142.0000,11460.176000
5,2014-06-01,3970.9140,9081.547667
6,2014-07-01,10032.9880,7715.300667
7,2014-08-01,7451.7740,7151.892000
8,2014-09-01,15507.7450,10997.502333
9,2014-10-01,12637.6780,11865.732333


## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
[PASTE ERROR MESSAGE and the exact SQL here]
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```

The error occurred because [YOUR_PROJECT] was left as a placeholder instead of the actual project ID (mgmt467project1). Replacing it with the valid project ID resolves the issue; as a quick sanity check, you can run SELECT COUNT(*) FROM \mgmt467project1.lab1dataset.superstore_clean`;` to confirm the table is accessible.

### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
[PASTE YOUR SQL HERE]
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

In [41]:
# Query 1: Slice 'Tables' by Region and Time (Year-Quarter)
query_string_region_time = """

WITH tables_only AS (
  SELECT
    Region,
    DATE_TRUNC(DATE(Order_Date), QUARTER) AS year_quarter,
    Sales,
    Profit,
    Discount
  FROM `mgmt467project1.lab1dataset.superstore_clean`
  WHERE Sub_Category = 'Tables'
),
agg AS (
  SELECT
    Region,
    year_quarter,
    SUM(Sales) AS total_sales,
    SUM(Profit) AS total_profit,
    SAFE_DIVIDE(SUM(Profit), NULLIF(SUM(Sales), 0)) AS profit_margin,
    AVG(Discount) AS avg_discount,
    COUNT(*) AS row_count
  FROM tables_only
  GROUP BY Region, year_quarter
)
SELECT *
FROM agg
ORDER BY Region, year_quarter;

"""

# Run the query
query_job = client.query(query_string_region_time)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)


,Region,year_quarter,total_sales,total_profit,profit_margin,avg_discount,row_count
0,Central,2014-01-01,2452.070,-89.1204,-0.036345,0.200000,3
1,Central,2014-04-01,1869.998,-640.7831,-0.342665,0.380000,5
2,Central,2014-07-01,1737.625,-276.9220,-0.159368,0.275000,4
3,Central,2014-10-01,1725.785,-417.5055,-0.241922,0.433333,3
4,Central,2015-01-01,102.438,-13.1706,-0.128571,0.300000,1
...,...,...,...,...,...,...,...
58,West,2016-10-01,11185.120,-984.0152,-0.087975,0.300000,13
59,West,2017-01-01,2776.394,-126.6095,-0.045602,0.220000,5
60,West,2017-04-01,1915.922,205.4001,0.107207,0.120000,5
61,West,2017-07-01,6757.998,-115.3883,-0.017074,0.190000,10


In [42]:
# Query 2: Control for Ship Mode within 'Tables'
query_string_ship_mode = """

WITH tables_only AS (
  SELECT
    Ship_Mode,
    DATE_TRUNC(DATE(Order_Date), QUARTER) AS year_quarter,
    Sales,
    Profit,
    Discount
  FROM `mgmt467project1.lab1dataset.superstore_clean`
  WHERE Sub_Category = 'Tables'
),
agg AS (
  SELECT
    Ship_Mode,
    year_quarter,
    SUM(Sales) AS total_sales,
    SUM(Profit) AS total_profit,
    SAFE_DIVIDE(SUM(Profit), NULLIF(SUM(Sales), 0)) AS profit_margin,
    AVG(Discount) AS avg_discount,
    COUNT(*) AS row_count
  FROM tables_only
  GROUP BY Ship_Mode, year_quarter
)
SELECT *
FROM agg
ORDER BY Ship_Mode, year_quarter;

"""

# Run the query
query_job = client.query(query_string_ship_mode)

# Convert to DataFrame
results_df = query_job.to_dataframe()

# Display
display(results_df)


,Ship_Mode,year_quarter,total_sales,total_profit,profit_margin,avg_discount,row_count
0,First Class,2014-04-01,602.6040,-24.9156,-0.041347,0.300000,2
1,First Class,2014-07-01,1622.4850,115.9120,0.071441,0.266667,3
2,First Class,2014-10-01,5028.4100,459.5571,0.091392,0.216667,6
3,First Class,2015-04-01,796.4250,-525.6405,-0.660000,0.500000,1
4,First Class,2015-07-01,975.8070,-394.6806,-0.404466,0.400000,3
5,First Class,2016-01-01,550.4310,-47.1798,-0.085714,0.300000,1
6,First Class,2016-04-01,2355.2760,185.7156,0.078851,0.200000,3
7,First Class,2016-07-01,161.2800,12.0960,0.075000,0.200000,1
8,First Class,2016-10-01,2508.6940,-267.4610,-0.106614,0.275000,4
9,First Class,2017-01-01,1559.9300,-4.9268,-0.003158,0.133333,3


To compare outcomes, place the region-time slice side by side with the ship-mode slice and look for patterns where profit outcomes diverge under similar discount levels. For example, if both East and West regions offered ~0.35 discounts but only East stayed unprofitable, then regional factors (like pricing or cost allocation) are at play. Likewise, if First Class and Same Day both had ~0.20 discounts yet Same Day shows losses while First Class is positive, then shipping cost structures—not discounting—drive the difference. By contrasting results across these dimensions, you can test whether discounts alone explain the negative margins, or whether profitability is contingent on where the sale occurred and how it was fulfilled

## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

Tables show high sales but swings from positive to deeply negative profit across regions, quarters, and ship modes, with losses reaching margins of -30% to -40% in certain East and Standard Class segments. These losses are likely driven by a combination of heavy discounting and high fulfillment costs, particularly in Same Day and Standard Class shipments. I recommend that the pricing team and operations jointly review discount thresholds and shipping policies for Tables in underperforming regions by the next quarter to align margin protection with service-level expectations. The key metric to monitor next is the profit margin by sub-category and ship mode over time, ensuring corrective actions lift unprofitable cells into at least break-even territory.

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

---
## Submission checklist
- [x] Kept prompts precise and reproducible  
- [x] Captured at least **one** CTE query and **one** window function query  
- [x] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [x] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---